# Probability Map Generator for Metal1D

Jupyter notebook to generate probability map file needed from Metal1D to perform predictions.

[**➡️ Link to Metal1D  Colab Notebook**](https://colab.research.google.com/github/lcbc-epfl/metal-site-prediction/blob/main/Metal1D/ColabMetal1D.ipynb).

*If using please cite:* 
>Accurate prediction of transition metal ion location via deep learning
>S.L. Dürr, A. Levy, U. Rothlisberger
>bioRxiv 2022.08.22.504853; doi: https://doi.org/10.1101/2022.08.22.504853

**Main steps:**
1. Analysis of LINK records from a list of PDB files provided by the user 
2. Statistical analysis of the data generating probability map

Note: Step 2 perform a basic filtering of the data
* Exclude non X-ray structures
* Exclude structures with resolution higher than a given threshold (default 2.5A)
* Maintain only the best-resolution structure among PDBs with the same protein name

The user is strongly suggested to perform some further *a priori* filtering, such as a clustering of structures at 30\% sequence identity to remove sequence and structural redundancy in the input dataset (for example, using [mmseqs2](https://www.nature.com/articles/nbt.3988)). 


**Output:**
* `scan_#` file: each line of this file contains information extracted from the scan, referred to one LINK
* `resultsALL_#` file: containing all the LINKs considered in the analysis, according to selection criteria
* `resultsLINK_#` file: containing the results of the LINK analysis (one-letter amino acid codes are used)
* `resultsCOORD_#` file: containing the results of the coordination analyis, extracted from the LINK records (one-letter amino acid codes are used)

Where `#` is the `LIGAND_ID` corresponding to the ligand analysed. 
**The last file is the probability map which needs to be provided to Metal1D to perform predictions.**

## 1. LINK scan
Scan of LINK records from PDB files provided by the user. 

Possible input formats:
* Array of PDB IDs, i.e. ['PDB1','PDB2',...] 
* External file in comma-separated format, i.e. PDB1, PDB2, ...

An easy way to obtain such a list is directly from the PDB website (define some search criteria -> Search -> Tabular Report: Entry IDs -> Download IDs).

As example [here](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_nonpolymer_instance_feature_summary.comp_id%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22ZN%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_nonpolymer_instance_feature_summary.type%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22HAS_COVALENT_LINKAGE%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_nonpolymer_instance_feature_summary.count%22%2C%22value%22%3A0%2C%22operator%22%3A%22equals%22%7D%7D%5D%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22exptl.method%22%2C%22value%22%3A%22X-RAY%20DIFFRACTION%22%2C%22operator%22%3A%22exact_match%22%7D%7D%5D%2C%22logical_operator%22%3A%22or%22%2C%22label%22%3A%22exptl.method%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22value%22%3A%7B%22from%22%3A0.5%2C%22to%22%3A1%2C%22include_lower%22%3Atrue%2C%22include_upper%22%3Afalse%7D%2C%22operator%22%3A%22range%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22value%22%3A%7B%22from%22%3A1%2C%22to%22%3A1.5%2C%22include_lower%22%3Atrue%2C%22include_upper%22%3Afalse%7D%2C%22operator%22%3A%22range%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22value%22%3A%7B%22from%22%3A1.5%2C%22to%22%3A2%2C%22include_lower%22%3Atrue%2C%22include_upper%22%3Afalse%7D%2C%22operator%22%3A%22range%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22value%22%3A%7B%22from%22%3A2%2C%22to%22%3A2.5%2C%22include_lower%22%3Atrue%2C%22include_upper%22%3Afalse%7D%2C%22operator%22%3A%22range%22%7D%7D%5D%2C%22logical_operator%22%3A%22or%22%2C%22label%22%3A%22rcsb_entry_info.resolution_combined%22%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22exptl.method%22%2C%22value%22%3A%22X-RAY%20DIFFRACTION%22%2C%22operator%22%3A%22exact_match%22%7D%7D%5D%2C%22logical_operator%22%3A%22or%22%2C%22label%22%3A%22exptl.method%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22entity_poly.rcsb_entity_polymer_type%22%2C%22value%22%3A%22Protein%22%2C%22operator%22%3A%22exact_match%22%7D%7D%5D%2C%22logical_operator%22%3A%22or%22%2C%22label%22%3A%22entity_poly.rcsb_entity_polymer_type%22%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22logical_operator%22%3A%22and%22%2C%22label%22%3A%22text%22%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22paginate%22%3A%7B%22start%22%3A0%2C%22rows%22%3A25%7D%2C%22results_content_type%22%3A%5B%22experimental%22%5D%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%2C%22scoring_strategy%22%3A%22combined%22%7D%2C%22request_info%22%3A%7B%22query_id%22%3A%2255f6ef3a71076a1bbf3cbb273aa48a3b%22%7D%7D) can be found a search for X-ray structures (resolution <2.5A), containing proteins and Comp ID ZN (ID for Zn2+ ion)

In [1]:
###################################
# CELL TO BE MODIFIED BY THE USER #
###################################
#
# Three input need to be provided
#

# 1) PDB IDs to analyse:
#   OPTION 1: Insert here array of PDB IDs
PDB_IDs = ['1LUG','1MWQ']  

#   OPTION 2: Read IDs from file (comma separated format)
ReadFromFile = False  # Change to True for Option 2
FileName = 'FILE.txt' # Change file name here   

# 2) Ligand name, i.e. name of the ligand to analyze involved in LINKs
LIGAND = 'ZN' # Change to LIGAND_ID for the desired metal

# 3) Chemical element
# used to exclude LINKs between atom of the ligand and the protein, but not involving the metal of interest 
ELEMENT = 'Zn'

In [2]:
# LINK scan - This cell only needs to be run with no changes 
import sys
import os
import math
import numpy as np
import pandas as pd
from urllib.request import urlopen

# Get PDB IDs from file
if(ReadFromFile):
    PDB_IDs = []
    InFile = open(FileName, 'r') #Input file = file name for the LIGAND ID in comma separated format
    for line in InFile:      
        currentline = line.replace('\n','').split(',')
        if(currentline[-1]==''):
            currentline = currentline[:-1]
        for i in range(0,len(currentline)):
            PDB_IDs.append(currentline[i].replace('\n',''))   

# Formatting check            
LIGAND = LIGAND.upper()   # PDB format contains upper case, changed in case user uses lower cases
ELEMENT = ELEMENT.upper() # PDB format contains upper case, changed in case user uses lower cases

# Define output file names
OutFile_linkscan = 'scan_'+LIGAND+'.txt'

# Scan of PDB IDs
scannedPDBs = 0
oldstdout = sys.stdout
sys.stdout.write('\n'+str(len(PDB_IDs))+' pdb structures present\n')

sys.stdout = OutFile = open(OutFile_linkscan, 'w')
for k in range(0,len(PDB_IDs)):
    try: #if PDB structure is found
         # URL corresponding to the pdb structure
         # check https://www.wwpdb.org/documentation/file-format-content/format33/v3.3.html for formattin information

        # General informations present in header read from here 
        #  Molecule IDs, chain names, Resolution
        url = 'https://files.rcsb.org/view/'+PDB_IDs[k]+'.pdb'
        PDB_url = urlopen(url)
            
        # Information related to the PDB entry
        PDB_entry = []    # Unique PDB code for the structure(PDB_ID)
        MOLECULE = []     # Name of the protein
        MolChain = []     # Different chains can be present in the pdb structure, corresponding to different proteins
                          #  and to different MOLECULE entries
        Experiment = []   # Experimental technique    
        Resolution = []   # Resolution

        for urlline in PDB_url:
            fileline = urlline.decode('utf-8')                        
            # Read PDB code from HEADER
            if(fileline[0:6]=='HEADER'):
                PDB_entry = fileline[62:66]
                # Read molecule name from COMPND 
                # Note: Synonyms not considered (present in SYNONYM section)
            if(fileline[0:6]=='COMPND'):
                if(fileline[11:20]=='MOLECULE:'):
                    MolChain.append(fileline[21:].replace('  ','').replace(';','').replace('\n','')) #each MOLECULE name is saved 
                if(fileline[11:17]=='CHAIN:'):
                    MolChain.append(fileline[18:].replace('  ','').replace(',','').replace(';','').replace('\n','')) #the corresponding chains are saved
            # Read experimental technique from EXPDTA                        
            if(fileline[0:6]=='EXPDTA'):
                Experiment = fileline[10:].replace('  ','').replace('\n','')                                      
            # Read experimental resolution from RESOLUTION
            if(fileline[0:22]=='REMARK   2 RESOLUTION.'):
                Resolution = fileline[23:30].replace(' ','')
        
        # Only X-RAY experiment considered
        #  according to selection criteria used
        if(Experiment != 'X-RAY DIFFRACTION '):
            break
                 
        # Structural information read from first biological assembly 
        url = 'https://files.rcsb.org/view/'+PDB_IDs[k]+'.pdb1'
        PDB_url = urlopen(url)
        # Information related to metallic res of interest
        LIGAND_atom = []         # Atom name
        LIGAND_altloc = []       # Alternate location indicator (if not present = empty)
        LIGAND_chain = []        # Chain identifier
        LIGAND_res_sequence = [] # Residue number
        LIGAND_element = []      # Chemical element
        # Biological residues linked to LIGAND
        LINKs_atom = []          # Atom name
        LINKs_altloc = []        # Alternate location indicator (if present)                
        LINKs_residue = []       # Residue name
        LINKs_chain = []         # Chain identifier
        LINKs_res_sequence = []  # Residue sequence number

        sys.stdout = OutFile = open(OutFile_linkscan, 'a')

        for urlline in PDB_url:
            fileline = urlline.decode('utf-8')                        
            if(fileline[0:4]=='LINK'):       
                # Each LINK record specifies a linkage
                #  only links where residue name == LIGAND are saved
                #  LINKs between the LIGAND residue are not considered
                if(fileline[47:50].replace(' ','')==LIGAND and fileline[17:20].replace(' ','')==LIGAND):
                    continue
                # LIGAND compared with both residues in each LINK
                # Case 1: LIGAND is the second residue of the LINK and the linked atom is the first    
                elif(fileline[47:50].replace(' ','')==LIGAND and fileline[17:20].replace(' ','')!=LIGAND):
                    LINKs_atom.append(fileline[12:16].replace(' ',''))
                    LINKs_altloc.append(fileline[16].replace(' ',''))                            
                    LINKs_residue.append(fileline[17:20].replace(' ',''))
                    LINKs_chain.append(fileline[21].replace(' ',''))
                    LINKs_res_sequence.append(fileline[22:26].replace(' ',''))
                    LIGAND_atom.append(fileline[42:46].replace(' ',''))
                    LIGAND_altloc.append(fileline[46].replace(' ',''))
                    LIGAND_chain.append(fileline[51].replace(' ',''))
                    LIGAND_res_sequence.append(fileline[52:56].replace(' ',''))       
                # Case 2: LIGAND is the first residue of the LINK and the linked atom is the second    
                elif(fileline[47:50].replace(' ','')!=LIGAND and fileline[17:20].replace(' ','')==LIGAND):
                    LIGAND_atom.append(fileline[12:16].replace(' ',''))
                    LIGAND_altloc.append(fileline[16].replace(' ',''))
                    LIGAND_chain.append(fileline[21].replace(' ',''))
                    LIGAND_res_sequence.append(fileline[22:26].replace(' ',''))
                    LINKs_atom.append(fileline[42:46].replace(' ',''))
                    LINKs_altloc.append(fileline[46].replace(' ',''))                            
                    LINKs_residue.append(fileline[47:50].replace(' ',''))
                    LINKs_chain.append(fileline[51].replace(' ',''))
                    LINKs_res_sequence.append(fileline[52:56].replace(' ',''))
            # When remarks are concluded the chains name/codes and the number of LINKs is fixed
            if(fileline[0:11]=='ATOM      1' or fileline[0:11]=='HETATM    1'):
                for i in range(0,len(LIGAND_atom)):
                    LIGAND_element.append('?')                        
                    for j in range(0,len(MolChain),2):                        
                        if(LIGAND_chain[i] in MolChain[j+1]):
                            MOLECULE.append(MolChain[j])                    
            if(fileline[0:4]=='ATOM' or fileline[0:6]=='HETATM'):
            # Consider lines corresponding to LIGAND linked atoms  
                if(fileline[12:16].replace(' ','') in LIGAND_atom and fileline[22:26].replace(' ','')  in LIGAND_res_sequence):                    #obtain the indeces corresponding to LINK atoms 
                    index = [i for i, j in enumerate(LIGAND_atom) if j == fileline[12:16].replace(' ','')] 
                    # For those atoms the element is saved 
                    # (because between LINK atoms it is possible to find elements different to the one of interest)            
                    if(LIGAND_element[index[0]]=='?'):
                        for i in index:
                            LIGAND_element[i] = fileline[76:78].replace(' ','')
        # Print results considering only the chemical element of interest        
        for i in range(0,len(LIGAND_atom)):
            if(LIGAND_element[i] == ELEMENT):
                sys.stdout.write(PDB_entry+';\t'+Experiment+';\t'+Resolution+';\t'+LIGAND_atom[i]+';\t'+LIGAND_altloc[i]+';\t'+LIGAND+';\t'+LIGAND_chain[i]+';\t'+LIGAND_res_sequence[i]+';\t'+LINKs_atom[i]+';\t'+LINKs_altloc[i]+';\t'+LINKs_residue[i]+';\t'+LINKs_chain[i]+';\t'+ LINKs_res_sequence[i]+';\t'+MOLECULE[i]+';\n')
        sys.stdout = oldstdout
        sys.stdout.write(str(k))                
        sys.stdout.write('\t'+PDB_IDs[k]+' scanning complete\n')
        # Number of correctly processed PDB files counted
        scannedPDBs = scannedPDBs+1
    except: # In some cases (extremely large strucutres) no PDB structure is found from PDB url
            # A warning is printed and the cycle proceeds with the other PDB_IDs             
        sys.stdout = oldstdout
        sys.stdout.write('\t'+PDB_IDs[k]+' not found - check PDB website\n')

sys.stdout = oldstdout 
sys.stdout.write('\n-----\n')
print(str(scannedPDBs)+' files successfully scanned, results in '+OutFile_linkscan+'\n')



2 pdb structures present
0	1LUG scanning complete
1	1MWQ scanning complete

-----
2 files successfully scanned, results in scan_ZN.txt



## 2. LINK analysis
Analysis of LINK records extracted from PDB files. 

A basic filtering of the data is applied, in order to:
* Exclude non X-ray structures
* Exclude structures with resolution higher than a given threshold (default 2.5A)
* Maintain only the best-resolution structure among PDBs with the same protein name

The user is strongly suggested to perform some further *a priori* filtering, such as a clustering of structures at 30\% sequence identity to remove sequence and structural redundancy in the input dataset (for example, using [mmseqs2](https://www.nature.com/articles/nbt.3988)). 

Two parameters can be set by the user for the analysis:
1. `RES_THRESHOLD`: the threshold resolution used to select X-ray structures (only structures with resolution `≤ RES_THRESHOLD` will be used). This step may be redundant if the list of PDB file list provided was already restricted to high resolution X-ray structures.
2. `DifferentChainOK`, which can be `True` (default) or `False`: if true all chains in the same pdb files are considered, if false only the first one. Testing for `ZN` showed that the probability maps were similar for both values, so `True` is used as default to have a larger statistics.

In [3]:
###################################
# CELL TO BE MODIFIED BY THE USER #
###################################
#
# Two parameters can be optionally 
# modified, default values are set
#

# 1) Threshold on resolution for the structures
RES_THRESHOLD = 2.5 #Threshold resolution for the analysis
                    #only X-ray structure with resolution RES_THRESHOLD or better are considered

# 2) Boolean flag, if true (default)
DifferentChainOK = True #True/False flag
                        #True if different chains in the same PDB can be considered
                        #False if only the first chain for each PDB is considered 

In [4]:
# LINK analysis - This cell only needs to be run with no changes

#3 letter codes for residues (HOH = water)
RES = ['HOH', 'ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

# Function to convert 3 letter code to 1 letter code (used to save coordination in a shorter format)
# Water molecules converted to 'O'
def res_1Letter(RES):
    RES_LetterCode = ['HOH', 'ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']
    R_LetterCode = ['O', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']    
    return R_LetterCode[RES_LetterCode.index(RES)]

# Functin for the LINK counting, where only the best resolution structure is considered 
def AddOKElement(list0, list1, list2, list3, list4, list5, list6, list7, list8, list9, list10, list11, list12, list13, OKel):
#append element to lists
    list0.append(OKel[0])
    list1.append(OKel[1])
    list2.append(OKel[2])
    list3.append(OKel[3])
    list4.append(OKel[4])
    list5.append(OKel[5])
    list6.append(OKel[6])
    list7.append(OKel[7])
    list8.append(OKel[8])
    list9.append(OKel[9])
    list10.append(OKel[10])
    list11.append(OKel[11])
    list12.append(OKel[12])
    list13.append(OKel[13].replace('-',' '))

# Function to remove entries corresponding to worst res pdb and appends the better one
def ReplaceOK(list0, list1, list2, list3, list4, list5, list6, list7, list8, list9, list10, list11, list12, list13, OKel, opt):
    if(opt == 'samespell'):    
        while(OKel[13] in list13):
            index = list13.index(OKel[13])
            list0.pop(index)
            list1.pop(index)
            list2.pop(index)
            list3.pop(index)
            list4.pop(index)
            list5.pop(index)
            list6.pop(index)
            list7.pop(index)
            list8.pop(index)
            list9.pop(index)
            list10.pop(index)
            list11.pop(index)
            list12.pop(index)
            list13.pop(index)
        AddOKElement(list0, list1, list2, list3, list4, list5, list6, list7, list8, list9, list10, list11, list12, list13, OKel)
    elif(opt == '-inline'):
        while(OKel[13] in list13):
            index = list13.index(OKel[13].replace('-',' '))
            list0.pop(index)
            list1.pop(index)
            list2.pop(index)
            list3.pop(index)
            list4.pop(index)
            list5.pop(index)
            list6.pop(index)
            list7.pop(index)
            list8.pop(index)
            list9.pop(index)
            list10.pop(index)
            list11.pop(index)
            list12.pop(index)
            list13.pop(index)
        AddOKElement(list0, list1, list2, list3, list4, list5, list6, list7, list8, list9, list10, list11, list12, list13, OKel)

# Function to append element to lists
def AddOKtoTOT(list0, list1, list2, list3, list4, list5, list6, list7, list8, list9, list10, list11, list12, list13, TOT_list0, TOT_list1, TOT_list2, TOT_list3, TOT_list4, TOT_list5, TOT_list6, TOT_list7, TOT_list8, TOT_list9, TOT_list10, TOT_list11, TOT_list12, TOT_list13):
    for i in range(0,len(list0)):
        TOT_list0.append(list0[i])
        TOT_list1.append(list1[i])
        TOT_list2.append(list2[i])
        TOT_list3.append(list3[i])
        TOT_list4.append(list4[i])
        TOT_list5.append(list5[i])
        TOT_list6.append(list6[i])
        TOT_list7.append(list7[i])
        TOT_list8.append(list8[i])
        TOT_list9.append(list9[i])
        TOT_list10.append(list10[i])
        TOT_list11.append(list11[i])
        TOT_list12.append(list12[i])
        TOT_list13.append(list13[i])

# Filtered lines saved here
TOT_PDB_entry = []
TOT_Experiment = []
TOT_Resolution = []
TOT_LABEL_atom = []
TOT_LABEL_altloc = []
TOT_LABEL = []
TOT_LABEL_chain = []
TOT_LABEL_res_sequence = []
TOT_LINKs_atom = []
TOT_LINKs_altloc = []
TOT_LINKs_residue = []
TOT_LINKs_chain = []
TOT_LINKs_res_sequence = []
TOT_MOLECULE = []
# For coordination analysis
# Total list scanned and coordination is stored
# results reported at the end considering all coordinations occurring
LIG_list = []
LIG_coord = []

# NAME used in the name of input files 
NAME = LIGAND

# 1st line output for LINK analysis
sys.stdout = OutFile = open('resultsLINK_'+NAME+'.txt', 'w')
sys.stdout.write('LIG/RES;')
for i in range(1, len(RES)):
    sys.stdout.write(res_1Letter(RES[i])+';')
sys.stdout.write('\n')

#analysis of all scan_LIGAND file provided
for i in range(1,len(sys.argv)-1):
    InFile = open('scan_'+NAME+'.txt', 'r')
    LIG_list.append((sys.argv[i].split('/')[-1].split('.')[0])[5:].upper())
    # OK_Data = data selected according to criteria
    # PDB_entry; Experiment; Resolution; LABEL_atom; LABEL_altloc; LABEL; LABEL_chain; LABEL_res_sequence; LINKs_atom; LINKs_altloc; LINKs_residue; LINKs_chain; LINKs_res_sequence; MOLECULE
    OK_PDB_entry = []
    OK_Experiment = []
    OK_Resolution = []
    OK_LABEL_atom = []
    OK_LABEL_altloc = []
    OK_LABEL = []
    OK_LABEL_chain = []
    OK_LABEL_res_sequence = []
    OK_LINKs_atom = []
    OK_LINKs_altloc = []
    OK_LINKs_residue = []
    OK_LINKs_chain = []
    OK_LINKs_res_sequence = []
    OK_MOLECULE = []
    # Filtering of data, accordin to criteria
    for line in InFile:
        fileline = line.replace('\t','').replace('\n','').split(';')[:-1]
        # Only X-RAY data 
        if(fileline[1]=='X-RAY DIFFRACTION '):
            # Only resolution RES_TRESHOLD or better
            if(float(fileline[2])<= RES_THRESHOLD):
                # Only LIGAND = RES (water + common amino acids) 
                if(fileline[10] in RES):
                    # .dat files present PDB entry in order    
                    # if PDB already present = acquiring one PDB entry in progress
                    if(fileline[0] in OK_PDB_entry):
                        # different chains in  the same PDB are considered                    
                        if(DifferentChainOK == True):
                            AddOKElement(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, fileline)
                        # only the first chain for each PDB is considered 
                        elif(DifferentChainOK == False):
                            if(fileline[11] == OK_LINKs_chain[OK_PDB_entry.index(fileline[0])]):
                                AddOKElement(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, fileline)
                    # if PDB entry not present = candidate for OK list
                    else:
                        # if same molecule already present -> Compare resolution and select the best
                        # case 1 = Exactly same spelling
                        if(fileline[13] in OK_MOLECULE):
                            if(float(fileline[2]) < float(OK_Resolution[OK_MOLECULE.index(fileline[13])])):
                                ReplaceOK(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, fileline, 'samespell')
                        # case 2 = same name, with a '-' 
                        #'-' in the possible new entry
                        elif((fileline[13]).replace('-',' ') in OK_MOLECULE):
                            if(float(fileline[2]) < float(OK_Resolution[OK_MOLECULE.index((fileline[13]).replace('-',' '))])):
                                ReplaceOK(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, fileline, '-inline')
                        # if same molecule NOT present -> addition
                        else:    
                            AddOKElement(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, fileline)

    InFile.close()
    AddOKtoTOT(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE, TOT_PDB_entry, TOT_Experiment, TOT_Resolution, TOT_LABEL_atom, TOT_LABEL_altloc, TOT_LABEL, TOT_LABEL_chain, TOT_LABEL_res_sequence, TOT_LINKs_atom, TOT_LINKs_altloc, TOT_LINKs_residue, TOT_LINKs_chain, TOT_LINKs_res_sequence, TOT_MOLECULE)
    # METAL-RESIDUE BINDING ANALYSIS
    # For each file (= each LIGAND)
    # LINKs for each RES are counted (excluding water)    
    RES_counter = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(0,len(OK_PDB_entry)):
        if(OK_LINKs_residue[i] != 'HOH'): #water excluded
            RES_counter[RES.index(OK_LINKs_residue[i])-1] += 1
    # output file    
    if(len(OK_LABEL)>0):
        sys.stdout.write(OK_LABEL[0]+';')
        for r in range(0,len(RES)-1):
            sys.stdout.write(str(RES_counter[r])+';')
        sys.stdout.write('\n')

    # COORDINATION ANALYSIS
    # OK entries sorted
    # lines containing information for a particular atom are subsequent 
    # (not always the case LINKs record of pdb structures)
    DataSet = pd.DataFrame(list(zip(OK_PDB_entry, OK_Experiment, OK_Resolution, OK_LABEL_atom,  OK_LABEL_altloc, OK_LABEL,  OK_LABEL_chain,  OK_LABEL_res_sequence, OK_LINKs_atom, OK_LINKs_altloc, OK_LINKs_residue, OK_LINKs_chain, OK_LINKs_res_sequence, OK_MOLECULE)), columns = ['PDB Entry','Experiment', 'Resolution', 'Metal Atom', 'Metal Altloc', 'Metal Ligand', 'Metal Chain', 'Metal ResSeq', 'Residue Atom', 'LINK Altloc', 'LINK RES', 'LINK Chain', 'LINK ResSeq', 'Molecule'])
    DataSet.sort_values(by=['PDB Entry', 'Metal Chain', 'Metal ResSeq'], inplace=True)
    
    OK_PDB_entry = DataSet['PDB Entry'].to_numpy()
    OK_Experiment = DataSet['Experiment'].to_numpy()
    OK_Resolution = DataSet['Resolution'].to_numpy()
    OK_LABEL_atom = DataSet['Metal Atom'].to_numpy()
    OK_LABEL_altloc = DataSet['Metal Altloc'].to_numpy()
    OK_LABEL = DataSet['Metal Ligand'].to_numpy()
    OK_LABEL_chain = DataSet['Metal Chain'].to_numpy()
    OK_LABEL_res_sequence = DataSet['Metal ResSeq'].to_numpy()
    OK_LINKs_atom = DataSet['Residue Atom'].to_numpy()
    OK_LINKs_altloc = DataSet['LINK Altloc'].to_numpy()
    OK_LINKs_residue = DataSet['LINK RES'].to_numpy()
    OK_LINKs_chain = DataSet['LINK Chain'].to_numpy()
    OK_LINKs_res_sequence = DataSet['LINK ResSeq'].to_numpy()
    OK_MOLECULE = DataSet['Molecule'].to_numpy()
    # informatin for last atom stored to compute coordination    
    prev_ID, prev_atom, prev_altloc, prev_chain, prev_res_sequence = [], [], [], [], []
    # vectors to store coordination + number of occurences
    coord, coord_num = [], []
    # current coordination built line per line and saved when atom changed
    current_coord = []
    for i in range(0,len(OK_PDB_entry)):
        # if metal atom informations change -> different atom from previous line -> coordination completed for previous atom
        if(OK_PDB_entry[i]!=prev_ID or OK_LABEL_atom[i]!=prev_atom or OK_LABEL_chain[i]!=prev_chain or OK_LABEL_res_sequence[i]!=prev_res_sequence):
            # case 1: different atom, without any altloc (possibility = '', 'A', 'B', ...)
            # if(len(OK_LABEL_altloc[i])==0 or (len(OK_LABEL_altloc[i])>0 and (prev_altloc=='' or prev_altloc ==[]))):
            if(OK_PDB_entry[i]!=prev_ID  or len(OK_LABEL_altloc[i])==0 or (len(OK_LABEL_altloc[i])>0 and (prev_altloc=='' or prev_altloc ==[]) or (OK_LABEL_res_sequence[i]!=prev_res_sequence))):
                # prev_ updated            
                prev_ID = OK_PDB_entry[i]
                prev_atom = OK_LABEL_atom[i]
                prev_altloc = OK_LABEL_altloc[i]
                prev_chain  = OK_LABEL_chain[i] 
                prev_res_sequence = OK_LABEL_res_sequence[i]
                # coordination saved
                if(current_coord != []):                
                    if(''.join(sorted(current_coord)) in coord):
                        coord_num[coord.index(''.join(sorted(current_coord)))] += 1
                    else:
                        coord.append(''.join(sorted(current_coord)))    
                        coord_num.append(1)                    
                # current coordination empty
                current_coord = []
                RES_altloc = ''
            # case 2: not emplty altloc -> 
            # if it is the first time for that atom, it can be followed by BABAB...
            # first memorized, others saved only if altloc is the first (ex. A)
            elif((OK_LABEL_altloc[i]!=prev_altloc and prev_altloc != []) or (OK_LABEL_chain[i]!=prev_altloc and prev_chain != [])):    
                continue
        # fill current coordination with RES
        # if more than one altloc is present for the metal or residue, only the first one is considered 
        if(len(RES_altloc)==0):
            RES_altloc = OK_LINKs_altloc[i]
        if((OK_LINKs_altloc[i]==RES_altloc or OK_LINKs_altloc[i]=='') and OK_LABEL_altloc[i] == prev_altloc):
            current_coord.append(res_1Letter(OK_LINKs_residue[i]))
    # last coordination saved here
    if(''.join(sorted(current_coord)) in coord):
        coord_num[coord.index(''.join(sorted(current_coord)))] += 1
    else:
        coord.append(''.join(sorted(current_coord)))    
        coord_num.append(1)
    # results in a list sorted according to coord_num (higher to lower)
    LIG_coord.append([x for _, x in sorted(zip(coord_num, coord), reverse=True)]) # row i-1 = coordination
    LIG_coord.append(sorted(coord_num, reverse=True))                             # row i = number of occurrences
    
# total result filled considering all coordinations observed
TOT_coord, TOT_coord_num = [], []    
for i in range(1,(len(sys.argv)-1)*2-1,2):
    for j in range(0,len(LIG_coord[i-1])):
        if(LIG_coord[i-1][j] in TOT_coord):
            TOT_coord_num[TOT_coord.index(LIG_coord[i-1][j])] += LIG_coord[i][j]
        elif(LIG_coord[i-1][j]!=''):        
            TOT_coord.append(LIG_coord[i-1][j])
            TOT_coord_num.append(LIG_coord[i][j])
TOT_coord = [x for _, x in sorted(zip(TOT_coord_num, TOT_coord), reverse=True)]
TOT_coord_num = sorted(TOT_coord_num, reverse=True)

# output file    
#    list of all coordination observed
#    total result summing all ligands
#    one line per each ligand
sys.stdout = OutFile = open('resultsCOORD_'+NAME+'.txt', 'w')
sys.stdout.write('LIG/COORD;')
for i in range(0, len(TOT_coord)):
    sys.stdout.write(TOT_coord[i]+';')
sys.stdout.write('\n')

k = 0
for i in range(1,(len(sys.argv)-1)*2-1,2):
    if(LIG_coord[i-1][0]!=''):
        sys.stdout.write(LIG_list[(k)]+';')
    for j in range(0,len(TOT_coord)):
        if(LIG_coord[i-1][0]!=''):
            if(TOT_coord[j] in LIG_coord[i-1][:]):
                sys.stdout.write(str(LIG_coord[i][LIG_coord[i-1][:].index(TOT_coord[j])])+';')
            else:
                sys.stdout.write(str(0)+';')
    if(LIG_coord[i-1][0]!=''):
        sys.stdout.write('\n')
    k += 1

DataSet = pd.DataFrame(list(zip(TOT_PDB_entry, TOT_Experiment, TOT_Resolution, TOT_LABEL_atom,  TOT_LABEL_altloc, TOT_LABEL,  TOT_LABEL_chain,  TOT_LABEL_res_sequence, TOT_LINKs_atom, TOT_LINKs_altloc, TOT_LINKs_residue, TOT_LINKs_chain, TOT_LINKs_res_sequence, TOT_MOLECULE)), columns = ['PDB Entry','Experiment', 'Resolution', 'Metal Atom', 'Metal Altloc', 'Metal Ligand', 'Metal Chain', 'Metal ResSeq', 'Residue Atom', 'LINK Altloc', 'LINK RES', 'LINK Chain', 'LINK ResSeq', 'Molecule'])
#Sort by multiple columns (by 1st, then by 2nd, and then by 3rd)
DataSet.sort_values(by=['PDB Entry', 'Metal Chain', 'Metal ResSeq'], inplace=True)
    
TOT_PDB_entry = DataSet['PDB Entry'].to_numpy()
TOT_Experiment = DataSet['Experiment'].to_numpy()
TOT_Resolution = DataSet['Resolution'].to_numpy()
TOT_LABEL_atom = DataSet['Metal Atom'].to_numpy()
TOT_LABEL_altloc = DataSet['Metal Altloc'].to_numpy()
TOT_LABEL = DataSet['Metal Ligand'].to_numpy()
TOT_LABEL_chain = DataSet['Metal Chain'].to_numpy()
TOT_LABEL_res_sequence = DataSet['Metal ResSeq'].to_numpy()
TOT_LINKs_atom = DataSet['Residue Atom'].to_numpy()
TOT_LINKs_altloc = DataSet['LINK Altloc'].to_numpy()
TOT_LINKs_residue = DataSet['LINK RES'].to_numpy()
TOT_LINKs_chain = DataSet['LINK Chain'].to_numpy()
TOT_LINKs_res_sequence = DataSet['LINK ResSeq'].to_numpy()
TOT_MOLECULE = DataSet['Molecule'].to_numpy()

sys.stdout = OutFile = open('resultsALL_'+NAME+'.txt', 'w')
for i in range(0, len(TOT_PDB_entry)):
    sys.stdout.write(TOT_PDB_entry[i]+';\t'+ TOT_Experiment[i]+';\t'+ TOT_Resolution[i]+';\t'+ TOT_LABEL_atom[i]+';\t'+ TOT_LABEL_altloc[i]+';\t'+ TOT_LABEL[i]+';\t'+ TOT_LABEL_chain[i]+';\t'+ TOT_LABEL_res_sequence[i]+';\t'+ TOT_LINKs_atom[i]+';\t'+ TOT_LINKs_altloc[i]+';\t'+ TOT_LINKs_residue[i]+';\t'+ TOT_LINKs_chain[i]+';\t'+ TOT_LINKs_res_sequence[i]+';\t'+ TOT_MOLECULE[i]+';\n')

The above procedures generated different ouptu files:
* `scan_#` file: each line of this file contains information extracted from the scan, referred to one LINK
* `resultsALL_#` file: containing all the LINKs considered in the analysis, according to selection criteria
* `resultsLINK_#` file: containing the results of the LINK analysis (one-letter amino acid codes are used)
* `resultsCOORD_#` file: containing the results of the coordination analyis, extracted from the LINK records (one-letter amino acid codes are used)

Where `#` is the `LIGAND_ID` corresponding to the ligand analysed.

**The last file is the probability map which needs to be provided to Metal1D to perform predictions.** [**➡️ Link to Metal1D  Colab Notebook**](https://colab.research.google.com/github/lcbc-epfl/metal-site-prediction/blob/main/Metal1D/ColabMetal1D.ipynb).
